## Train the model

In [1]:
from utils import dataloader
import utils.model
from datetime import datetime

filter_timestamp = cutoff_timestamp = None#int(datetime(2021, 2, 19, 0).timestamp())
use_user_index = "user_index.parquet"#"train_user_index.parquet"
csv_data_location = "downloaded_data"

In [2]:
dl = dataloader.RecSys2021TSVDataLoader(csv_data_location, use_user_index, mode="train", filter_timestamp=filter_timestamp)

In [3]:
%%time
data = next(iter(dl))
train_data = data[0]
labels = data[1]

CPU times: user 3min 9s, sys: 8.77 s, total: 3min 18s
Wall time: 3min 22s


In [4]:
recsysxgb = utils.model.RecSysXGB1()
xgb_params = {'objective': 'binary:logistic', 'eval_metric':'map'}
recsysxgb.train_in_memory(train_data, labels, xgb_params, save_dir = "xgb_models_05_submission")

## Evaluate the model

In [9]:
#recsysxgb = utils.model.RecSysXGB1("xgb_models_04_better_features1")
dl = dataloader.RecSys2021TSVDataLoader(csv_data_location, use_user_index, mode="val", filter_timestamp=filter_timestamp)

In [10]:
res = recsysxgb.evaluate_validation_set(dl)
for (target, ap, rce) in zip(recsysxgb.targets__, res[0], res[1]):
    print(f"{target}: {ap} - {rce}")

has_reply: 0.16962196226086176 - 10.747179145045871
has_retweet: 0.38669954726459954 - 19.86339784995025
has_retweet_comment: 0.043576703594668656 - -20.327808991302177
has_like: 0.6932498419910613 - 9.419793147661126


In [20]:
dict(sorted(recsysxgb.clfs_["has_like"].get_score(importance_type='gain').items(), key=lambda item: item[1],reverse=True))

{'TE_like_a_A': 17684.535543281632,
 'TE_retweet_b_B': 3859.1417743324137,
 'n_like_a_A': 3752.7517747058814,
 'type_encoding': 2855.152270493333,
 'n_like_b_B': 2604.5063219985723,
 'n_present_a_A': 2312.108605638519,
 'n_present_b_A': 1986.8852737500001,
 'a_follower_count': 1946.3226197428571,
 'TE_reply_b_B': 1858.9121773749998,
 'n_retweet_b_A': 1570.3047206769234,
 'bert_token_len': 1454.75,
 'n_photos': 1375.9783322888889,
 'TE_like_b_B': 1352.4801489709928,
 'b_creation_delta': 1246.4472216285715,
 'n_reply_a_A': 1227.1597588233333,
 'a_follows_b': 1008.1858988,
 'TE_retweet_a_B': 957.880188,
 'n_day_45_A': 826.6675516133333,
 'a_is_verified': 775.8146670000001,
 'n_retweet_comment_a_A': 723.8681656666668,
 'TE_retweet_a_A': 680.0563918333332,
 'n_day_40_A': 617.94104,
 'n_retweet_b_B': 520.4279785,
 'b_following_count': 452.9670416666666,
 'n_retweet_a_A': 255.85770425,
 'n_present_b_B': 232.654556,
 'n_day_36_B': 67.7659912,
 'TE_like_a_B': 26.8093872,
 'n_retweet_a_B': 24.71

## Try sample test run

In [ ]:
import utils.model
import utils.dataloader

dl = utils.dataloader.RecSys2021TSVDataLoader("test", "user_index.parquet", mode="test", load_n_batches=-1)
recsysxgb = utils.model.RecSysXGB1("xgb_models_05_submission")

recsysxgb.evaluate_test_set(testLoader = dl, output_file = "res.csv")

## Other

In [2]:
from utils import dataloader
import utils.model
from datetime import datetime

filter_timestamp = cutoff_timestamp = None#int(datetime(2021, 2, 19, 0).timestamp())
use_user_index = "user_index.parquet"#"train_user_index.parquet"
csv_data_location = "downloaded_data"

In [3]:
dl = dataloader.RecSys2021TSVDataLoader(csv_data_location, use_user_index, mode="train", filter_timestamp=filter_timestamp, load_n_batches=1, batch_size=5000000, verbose=1)

In [3]:
r = [a[0] for a in dl]

Finished Batch Nr. 1 from file part-00000.tsv!
Finished Batch Nr. 2 from file part-00000.tsv!
Finished Batch Nr. 3 from file part-00000.tsv!
Finished Batch Nr. 4 from file part-00000.tsv!
Finished Batch Nr. 5 from file part-00000.tsv!
Finished Batch Nr. 6 from file part-00000.tsv!
Finished Batch Nr. 7 from file part-00000.tsv!
Finished Batch Nr. 8 from file part-00000.tsv!
Finished Batch Nr. 9 from file part-00000.tsv!
Finished Batch Nr. 10 from file part-00000.tsv!


In [4]:
r[0]

,a_follower_count,a_following_count,a_is_verified,b_follower_count,b_following_count,b_is_verified,a_follows_b,bert_token_len,n_photos,n_videos,...,TE_like_b_A,TE_like_b_B,TE_retweet_a_A,TE_retweet_a_B,TE_retweet_b_A,TE_retweet_b_B,TE_retweet_comment_a_A,TE_retweet_comment_a_B,TE_retweet_comment_b_A,TE_retweet_comment_b_B
0,526918,163,False,220,638,False,False,3,0.693147,0.000000,...,0.655940,0.582249,0.040542,0.050395,0.158352,0.006139,0.001334,0.005083,0.011672,0.000292
1,3062,1695,False,392,487,False,True,3,-1.000000,-1.000000,...,0.718410,0.644231,0.020698,0.055194,0.173433,0.053160,0.019945,0.005567,0.012784,0.001226
2,2298,346,False,1306,5000,False,True,2,-1.000000,-1.000000,...,0.285400,0.757480,0.035474,0.074269,0.031520,0.103158,0.000508,0.002088,0.007086,0.005966
3,4813450,662,True,90,91,False,False,3,0.693147,0.000000,...,0.754331,0.705587,0.031772,0.052660,0.132105,0.148972,0.004921,0.002851,0.013423,0.012081
4,2740151,15255,True,158,411,False,False,3,-1.000000,-1.000000,...,0.754331,0.803464,0.016482,0.057954,0.132105,0.105684,0.002477,0.005845,0.013423,0.010738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,70817,847,False,9,330,False,False,3,0.693147,0.000000,...,0.731210,0.776664,0.045440,0.057954,0.120095,0.120095,0.005333,0.005845,0.012203,0.012203
996,20974,523,True,19,301,False,False,3,-1.000000,-1.000000,...,0.766029,0.718410,0.034596,0.057954,0.125814,0.173433,0.005546,0.005845,0.012784,0.012784
997,2952,675,False,31,72,False,True,4,0.000000,0.000000,...,0.522638,0.856602,0.051763,0.046363,0.096710,0.044636,0.003891,0.004676,0.005593,0.002581
998,29819,2,False,9,162,False,False,4,0.000000,0.693147,...,0.655940,0.701054,0.203735,0.057954,0.158352,0.014927,0.001872,0.005845,0.011672,0.001517
